In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from pathlib import Path

pio.renderers.default = "browser"
# pd.options.plotting.backend = 'plotly'

This is a data science project about League of Legends for the class DSC 80 at UCSD. 

Names: Joyce Hu and Ellie Wang

# Introduction

For our project, we are interested in exploring dataset about the game League of Legends from Oracle's Elixir. As League of Legends continues to constantly update the game, there are changes to all aspects that we want to investigate. More specifically, we aim to answer qusetions about the champion Neeko.

# Data Cleaning and Exploratory Data Analysis

We created two datasets: one that addresses our hypothesis question and one that relates to our classification model. For both of these datasets, we impute team statistics into missing player level stats for `firstdragon`, `firstherald`, `heralds`, `opp_heralds`


In [2]:
fp1 = '../data/league_2021.csv'
fp2 = '../data/league_2022.csv'
fp3 = '../data/league_2023.csv'
fp4 = '../data/league_2024.csv'

leeg21 = pd.read_csv(fp1)
leeg22 = pd.read_csv(fp2)
leeg23 = pd.read_csv(fp3)
leeg24 = pd.read_csv(fp4)

league = pd.concat([leeg21, leeg22, leeg23, leeg24])
league.head()

/Users/elliekwang/miniforge3/envs/dsc80/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning:

Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.



,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,118.0,748.0,-56.0,-4.0,2.0,0.0,1.0,0.0,1.0,1.0
1,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,84.0,2120.0,3405.0,62.0,3.0,2.0,0.0,2.0,0.0,3.0
2,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,143.0,1578.0,354.0,15.0,2.0,3.0,0.0,0.0,0.0,0.0
3,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,120.0,124.0,102.0,10.0,0.0,4.0,2.0,1.0,1.0,2.0
4,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,25.0,448.0,450.0,3.0,1.0,4.0,0.0,0.0,2.0,2.0


## Data Cleaning

In [70]:
# make copy 
pre_cleaning = league.copy()

# binary variable for missing patch
league['patch_missing'] = (league['patch'].isna()).astype(int)

## impute missing patches
patches = {
    '2022-08-11 08:37:33':12.15, 
    '2022-08-12 06:08:39':12.15, 
    '2022-08-12 07:04:40':12.15, 
    '2022-08-12 07:52:33':12.15, 
    '2022-08-12 08:51:49':12.15, 
    '2022-08-12 09:58:13':12.15, 
    '2022-08-13 06:08:31':12.15, 
    '2022-08-13 06:55:35':12.15, 
    '2022-08-13 06:08:31':12.15,
    '2022-08-13 06:55:35':12.15,
    '2022-08-13 07:47:42':12.15,
    '2023-04-05 09:17:28':13.07,
    '2023-04-05 10:18:03':13.07, 
    '2023-04-05 11:23:10':13.07,
    '2023-04-05 12:21:13':13.07,
    '2023-04-05 13:10:18':13.07,
    '2023-05-08 06:12:19':13.09,
    '2023-05-08 07:07:13':13.09,
    '2023-05-08 08:12:21':13.09,
    '2023-05-08 09:06:19':13.09,
    '2023-05-08 10:08:36':13.09,
    '2024-02-21 06:02:57':14.03,
    '2024-02-21 06:56:52':14.03, 
    '2024-02-21 07:43:27':14.03,
    '2024-02-21 08:31:33':14.03, 
    '2024-02-21 09:17:02':14.03
}
league['patch'] = league['patch'].fillna(league['date'])
league['patch'] = league['patch'].replace(patches)

## impute team statistics into missing player level stats for 'firstdragon', 'firstherald', 'heralds', 'opp_heralds'

# get team summary statistics 
team_stats = (league[(league['participantid'] == 100) | (league['participantid'] == 200)]
              .loc[:,['gameid', 'teamid','firstdragon', 'firstherald', 'heralds', 'opp_heralds']])

# subset dataframe to player level data
league = league[(league['participantid'] != 100) & (league['participantid'] != 200)]

# drop stats cols in league to retain col names after merge
league = league.drop(columns=['firstdragon', 'firstherald', 'heralds', 'opp_heralds'])

# merge team stats onto player level data
league = league.merge(team_stats, on=['gameid', 'teamid'], how='left')

# create a binary variable for before and after Patch 13.9
league['post_rework'] = (league['patch'] >= 13.09).astype(int)

# create a binary variable for whether neeko played in the game
league['neeko'] = (league['champion'] == 'Neeko').astype(int)

# create a new variable for difference in heralds gained 
league['herald_diff'] = league['heralds'] - league['opp_heralds']

# keep features we need
league = (league.loc[:, ['gameid', 'teamid', 'neeko', 'position', 'datacompleteness','league', 
                'patch', 'firstdragon', 'firstherald', 'herald_diff', 'goldat15','xpat15','csat15',
                'golddiffat15','xpdiffat15','csdiffat15','killsat15','assistsat15','deathsat15',
                'result']])

league.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_assistsat15,opp_deathsat15,patch_missing,firstdragon,firstherald,heralds,opp_heralds,post_rework,neeko,herald_diff
0,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,1.0,1.0,0,1.0,1.0,2.0,0.0,0,0,2.0
1,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,0.0,3.0,0,1.0,1.0,2.0,0.0,0,0,2.0
2,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,0.0,0.0,0,1.0,1.0,2.0,0.0,0,0,2.0
3,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,1.0,2.0,0,1.0,1.0,2.0,0.0,0,0,2.0
4,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,2.0,2.0,0,1.0,1.0,2.0,0.0,0,0,2.0


## Univariate Analysis

## Bivariate Analysis

## Interesting Aggregates

This is a pivot table of the number of games with neeko from patch 12.10 to 14.05, seperated by the position she was played in. It is interesting because we can see the amount of games she was played in increases over time. 

In [91]:
# just looking at patch 12.10 - present
recent = league[league['patch'] >= 12.10]

(recent.pivot_table(index='patch', columns='position', values='neeko', aggfunc='sum', 
                    margins=True, margins_name='total'))




position,bot,jng,mid,sup,top,total
patch,,,,,,
12.1,0,0,0,0,1,1
12.11,0,0,1,0,0,1
12.12,1,0,0,0,0,1
12.13,0,0,1,3,0,4
12.14,0,0,1,0,0,1
12.15,0,0,2,0,0,2
12.16,0,1,0,0,2,3
12.17,0,0,0,0,0,0
12.18,0,0,0,0,0,0


In [93]:
league['patch_missing']

0         0
1         0
2         0
3         0
4         0
         ..
382835    0
382836    0
382837    0
382838    0
382839    0
Name: patch_missing, Length: 382840, dtype: int64

# Assessment of Missingness

## NMAR Analysis

We do not belive that there is a column in our dataset that is NMAR. While there are a lot of columns with missing data, we believe that most are related to the column `datacompletedness`. 

Additionally, based on our preliminary findings when looking at the "patch" column that has missing data, we believe that the column "league", which is composed of the different leagues played in the game, could explain the missingness since we found that there were only missing patches for two leagues out of the 56.

## Missingness Dependency: `patch`

A column that is important to our hypothesis test is the `patch` column. However, we found that the `patch` column had **382840** missing values. We analyzed its relationship with the `league` and `result` columns to check its missingness dependency. 

### `league`

In [110]:
piv = league.pivot_table(index='league', columns='patch_missing', aggfunc='size').fillna(0)
piv = piv / piv.sum()
obs = piv.diff(axis=1).iloc[:,-1].abs().sum() / 2
print(obs)
piv.head()

0.8441191845269211


patch_missing,0,1
league,,
AL,0.005776,0.0
AOL,0.000967,0.0
BIG,0.003868,0.0
BL,0.002692,0.0
BM,0.002300,0.0


In [95]:
n_repetitions = 500
shuffled = league.loc[:,['league', 'patch_missing']]

tvds = []
for _ in range(n_repetitions):
    
    # shuffle whether patch is missing
    shuffled['shuffle_patch'] = np.random.permutation(shuffled['patch_missing'])
    
    # Computing and storing the TVD.
    pivoted = (
        shuffled.pivot_table
        (index='league', columns='shuffle_patch', aggfunc='size').fillna(0)
    )
    
    pivoted = pivoted / pivoted.sum()
    
    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)
tvds[:10]

[0.17497734797002962,
 0.16664052970900856,
 0.21040730092350585,
 0.2060433002265203,
 0.21202474298658303,
 0.19888177382819308,
 0.23489763024917232,
 0.19641226694546088,
 0.22607771388743683,
 0.21255706569088692]

#### p-value: 

In [100]:
(np.array(tvds) >= obs).mean()

0.0

### `result`

In [109]:
piv = league.pivot_table(index='result', columns='patch_missing', aggfunc='size').fillna(0)
piv = piv / piv.sum()
obs = piv.diff(axis=1).iloc[:,-1].abs().sum() / 2

print(obs)
piv

0.00014375326711968395


patch_missing,0,1
result,,
0,0.500144,0.5
1,0.499856,0.5


In [104]:
n_repetitions = 500
shuffled = league.loc[:,['result', 'patch_missing']]

tvds = []
for _ in range(n_repetitions):
    
    # shuffle whether patch is missing
    shuffled['shuffle_patch'] = np.random.permutation(shuffled['patch_missing'])
    
    # Computing and storing the TVD.
    pivoted = (
        shuffled.pivot_table
        (index='result', columns='shuffle_patch', aggfunc='size').fillna(0)
    )
    
    pivoted = pivoted / pivoted.sum()
    
    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)
tvds[:10]

/var/folders/p3/nfh6l6hn58q316g9ty88_q9r0000gn/T/ipykernel_26619/2255443645.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[0.03737976999477258,
 0.012364087820177727,
 0.054344397978741965,
 0.024871928907475166,
 0.00014375326711968395,
 0.004313033629552154,
 0.03766727652901203,
 0.020702648545042723,
 0.025159435441714617,
 0.029041209269907636]

In [105]:
(np.array(tvds) >= obs).mean()

1.0

In [108]:
piv.plot(kind='barh', title='Gender by Missingness of Child Height (MAR Example)', barmode='group')
